# FLUORIMETRIA: spettro emissione

si hanno soluzioni di rodamina in acqua consequenzialmente a concentrazioni minori

OBIETTIVI:
viene variata la concentrazione di rodamina. Si vuole mettere in relazione l'assorbanza di rodamina con l'intensità di fluorescenza

$\epsilon$ = coefficiente estinzione molare rodamina 6G = $116000 M^{-1} cm^{-1} $ 

$A = \epsilon C l$

In [1]:
# import librerie
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import os
import sys
import plotly.graph_objects as go
dir_path = os.path.abspath('')
sys.path.append(dir_path + '/../')
from labbiofisica import Interpolazione, final_val
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d

Assorbanza:

Segue il calcolo delle assorbanze, in laboratorio è stato usato lo spettrofotometro per misurare l'assorbanza del campione iniziale. In seguito la soluzione è stata diluita usando $A_i V_i = A_f V_f$ (valida in quanto $C \propto A$)

In [2]:
# definizione di costanti e funzione di interpolazione parabolica
λexc = 526 #nm frequenza di eccitazione
SIGMA_LAMBDA = 1.5 #nm DICHIARATI DAL COSTRUTTORE

A0 = 0.4800416799849726
sigmaA0 = 0.0003353431037236748

sigmaV = 1 # μL, comodo perchè si ha sempre a che fare con rapporti

sigmaySPETTROFLUORIMETRO = 0.0045 # percent 0.45% valore preso da STUDIO_ERRORI_SPETTROFLUORIMETRO.ipynb

# Extracting the relevant columns for plotting
WAVELENGHTS = ['λ 5', 'λ 4', 'λ 3', 'λ 2', 'λ 1', 'λ 0.8', 'λ 0.6', 'λ 0.4', 'λ 0.2', 'λ 0.1']
INTENSITIES = ['I 5', 'I 4', 'I 3', 'I 2', 'I 1', 'I 0.8', 'I 0.6', 'I 0.4', 'I 0.2', 'I 0.1']
# CONCENTRATIONS=[5,4,3,2,1,0.8,0.6,0.4,0.2,0.1]

# FUNZIone per estrarre i massimi con fit parabolico
def max_fit_parabolic(x, λcenter, a, IMAX): # -a*(x-λcenter)**2 + IMAX
    return -a*(x-λcenter)**2 + IMAX

In [3]:
# Caricamento del file AAT bioquest
# # real rodamina 
# R6G_emission = pd.read_csv(r'.\data\Rhodamine 6G_emission.csv', sep=',')
# R6G_absorbance = pd.read_csv(r'.\data\Rhodamine 6G_absorbance.csv', sep=',')
# R6G_excitation = pd.read_csv(r'.\data\Rhodamine 6G_excitation.csv', sep=',')

I valori di assorbanza iniziale sono calcolati in $concentrazione\_iniziale.ipynb$


determinazione di tutte le concentrazioni di cui è stato fatto lo spettro di fluorescenza:

in laboratorio è stata raccolta la misura di:

- $V_{in}$ = volume estratto dalla soluzione precedente
- $V_{fin}$ = volume estratto dalla soluzione precedente + tampone (acqua)

In [4]:
# caricamento valori sperimentali delle soluzioni (vi, vf)
filename = './data/concentrazioni_rodamina.csv'

volumi = pd.read_csv(filename,sep=',')
volumi = volumi[['Vi (muL)','Vf (muL)']]
# Display the DataFrame horizontally
display(volumi.T)

,0,1,2,3,4,5,6,7,8
Vi (muL),2210,2252,2000,1511,2423,2251,2002,1497,1504
Vf (muL),3008,3015,3015,3016,3010,3002,3000,2994,3003


In [5]:
# calcolo valori delle assorbanze
#Cf Vf = Ci Vi -> Cf = Ci Vi / Vf
# sigma V = 1 μL
ASS = [A0]
SIGMA_ASS = [sigmaA0]

for idx, row in volumi.iterrows():
    vi, vf = row['Vi (muL)'], row['Vf (muL)']
    An = ASS[idx] * vi / vf
    sAn = An * np.sqrt((sigmaV / vi)**2 + (sigmaV / vf)**2 + (SIGMA_ASS[idx] / ASS[idx])**2)
    ASS.append(An)
    SIGMA_ASS.append(sAn)

ASS = np.array(ASS)
SIGMA_ASS = np.array(SIGMA_ASS)

# Create a pandas DataFrame to display ASS and SIGMA_ASS
assorbance_df = pd.DataFrame({
    'A': ASS,
    'σA': SIGMA_ASS
})
display(assorbance_df.T)

,0,1,2,3,4,5,6,7,8,9
A,0.480042,0.352690,0.263436,0.174750,0.087549,0.070475,0.052845,0.035265,0.017633,0.008831
σA,0.000335,0.000316,0.000278,0.000212,0.000124,0.000107,0.000085,0.000061,0.000033,0.000018


ora che sono noti i valori delle assorbanze dei vari campioni di cui è stata fatto lo spettro di fluorescenza si importano i veri e propri spettri di fluorescenza

In [6]:
# Caricamento spettri allo spettrofluorimetro

filename = 'data/assorbanza_rodamina_concentrazioni_tot.csv'

header = ['λ 5','I 5','λ 4','I 4','no λ 4','no I 4','λ 3','I 3','λ 2','I 2','λ 1','I 1','λ 0.8','I 0.8','λ 0.6','I 0.6','λ 0.4','I 0.4','λ 0.2','I 0.2','λ 0.1','I 0.1']

# fondo = pd.read_csv('./data/fondoH20Sspettrofluorimetro.csv',sep=',',header=1,nrows=150)[['Wavelength (nm).1','Intensity (a.u.).1']]
# fondo.columns = ['λfondo','Ifondo']
# fondo.tail()

# from scipy.interpolate import interp1d
# fondo_interp = interp1d(fondo['λfondo'], fondo['Ifondo'])

data = pd.read_csv(filename,sep=',',header=1,nrows=117)
data = data.iloc[:, :-1] # drop last column
data.columns = header

# APPEND ERRORS ON THE PEAK:
for i, (λ_col, I_col) in enumerate(zip(WAVELENGHTS, INTENSITIES)):
    errors = data[I_col].copy() * sigmaySPETTROFLUORIMETRO
    maxid = data[I_col].idxmax()
    center = data[λ_col][maxid]
    errors = errors[(data[λ_col] < center+6) & (data[λ_col] > center-6)] #  6 è un buffer, sperimentalmente il valore è affisabile solo +- 5nm
    name = 'σ ' + I_col
    data[name] = errors 

# Save the modified data to a new CSV file


plot spettri di fluo

In [15]:
fig = go.Figure()

colors = px.colors.sequential.Viridis_r
for i, (λ_col, I_col, a) in enumerate(zip(WAVELENGHTS, INTENSITIES, ASS)):
    fig.add_trace(go.Scatter(
        x=data[λ_col], 
        y=data[I_col],
        mode='lines',
        name=f'{np.round(a, 3)}',
        line=dict(color=colors[i % len(colors)]),  # Ensure colors cycle if more traces than colors
        error_y=dict(
            type='data',
            array=data[f'σ {I_col}'],  # Add error bars using the corresponding error column
            visible=True
        )
    ))

fig.update_layout(
    xaxis_title='Wavelength (nm)',
    yaxis_title='Intensity (a.u.)',
    height=600,
    width=800,
    yaxis=dict(range=[0, 1000]),
    legend=dict(
        title=dict(text='Assorbance'),
        x=1,  # Position legend at the far right
        y=1,  # Position legend at the top
        xanchor='right',  # Anchor legend to the right
        yanchor='top'     # Anchor legend to the top
    ),
    font=dict(size=14)
)

fig.write_html(dir_path + r"\html\emission_spectrum.html")
fig.write_image(dir_path + r"\images\emission_spectrum.png")
fig.show()

NOTA: i picchi non sono centrati

segue fit parabolico dei picchi dello spettro, vengono considerati solo i 5 punti a destra e 5 a sinistra del punto massimo dello spettro

segue l'estrapolazione del valore di Imax

In [8]:
λcenter_list = []
a_list = []
IMAX_list = []
error_λcenter_list = []
error_a_list = []
error_IMAX_list = []

for λ_col, I_col in zip(WAVELENGHTS, INTENSITIES):
    Λ = data[λ_col]
    I = data[I_col]
    σI = data[f'σ {I_col}']
    Λmaxcenter = I.idxmax()

    λcenter = Λ[Λmaxcenter] # guess
    IMAX = I.max() # guess
    a = 1 # guess

    xrangemax = Λ[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max
    yrangemax = I[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max
    σIrangemax = σI[Λmaxcenter-5:Λmaxcenter+5] # 10 points around the max

    interp = Interpolazione(xrangemax, yrangemax, σIrangemax,max_fit_parabolic, [λcenter, a, IMAX], names=['λcenter', 'a', 'IMAX'])
    λcenter, a, IMAX = interp.values.values()
    error_λcenter, error_a, error_IMAX = interp.errors.values()
    λcenter_list.append(λcenter)
    a_list.append(a)
    IMAX_list.append(IMAX)
    error_λcenter_list.append(error_λcenter)
    error_a_list.append(error_a)
    error_IMAX_list.append(error_IMAX)

# # to numpy
λcenter_list = np.array(λcenter_list)
a_list = np.array(a_list)
IMAX_list = np.array(IMAX_list)
error_λcenter_list = np.array(error_λcenter_list)
error_a_list = np.array(error_a_list)
error_IMAX_list = np.array(error_IMAX_list)

print('Tabella con i fit di tutti i picchi delle parabole')
max_fit_parabolic_dataframe = pd.DataFrame({'Absorbance': ASS, 'λcenter': λcenter_list, 'a': a_list, 'IMAX': IMAX_list, 'error_λcenter': error_λcenter_list, 'error_a': error_a_list, 'error_IMAX': error_IMAX_list})
display(max_fit_parabolic_dataframe)

Tabella con i fit di tutti i picchi delle parabole


,Absorbance,λcenter,a,IMAX,error_λcenter,error_a,error_IMAX
0,0.480042,553.682999,2.593461,941.117146,0.088086,0.177170,1.998978
1,0.352690,553.605458,1.800572,816.838393,0.109900,0.155299,1.743310
2,0.263436,552.772023,1.743935,718.458398,0.103033,0.137340,1.528068
3,0.174750,551.924028,1.079676,493.259725,0.119787,0.094140,1.042303
4,0.087549,551.462054,0.469150,238.411966,0.122389,0.045713,0.511283
5,0.070475,551.469740,0.375982,170.297252,0.144500,0.033035,0.350329
6,0.052845,551.244523,0.240518,117.718777,0.142487,0.022860,0.246519
7,0.035265,551.046902,0.103811,52.557822,0.137406,0.010217,0.111394
8,0.017633,550.904438,0.032285,19.327347,0.154810,0.003714,0.041100
9,0.008831,551.032740,0.025758,11.341036,0.119088,0.002199,0.024014


In [9]:
center = np.mean(λcenter_list)
sigma = np.std(λcenter_list)
print('λcenter =', final_val(center, sigma, udm='nm'))
print('NOTA: sigmaLambda dichiarata dal costruttore vale 1.5nm e quindi è compatibile con il valore di sigma trovato')

λcenter = 551.91 ± 1.01 nm
NOTA: sigmaLambda dichiarata dal costruttore vale 1.5nm e quindi è compatibile con il valore di sigma trovato


The function $ F(C) $ is defined as:

$ F(C, F_0, k, y_0) = F_0 \cdot \left( 1 - e^{-k \cdot C} \right) + y_0$

where:
- $ C $ is the concentration,
- $ F_0 $ is the maximum fluorescence intensity,
- $ k $ is the rate constant,
- $ y_0 $ is the baseline fluorescence intensity.

In [10]:
# fit with the exponential 1-exp(-k*A)

def F_C_fit(A,F0,k,y0):
    return F0*(1-np.exp(-k*A)) + y0

In [11]:
start = 0
end = -2

I = np.array(IMAX_list[start:end])
sigmaI = np.array(error_IMAX_list[start:end])

A = np.array(ASS[start:end]) # convert
sigmaA = np.array(SIGMA_ASS[start:end]) # propagate error on A

# Create a scatter plot with error bars using Plotly
fig = go.Figure()

# Add data points with error bars
fig.add_trace(go.Scatter(
    x=A,
    y=I,
    error_y=dict(
        type='data',
        array=sigmaI,
        visible=True
    ),
    error_x=dict(
        type='data',
        array=sigmaA,
        visible=True
    ),
    mode='markers',
    name='Data with error'
))

fig.update_layout(
    xaxis=dict(
        title='Absorbance',
        # type='log'  # Set x-axis to log scale
    )
)

# Update layout for labels
fig.update_layout(
    xaxis=dict(
        title='Absorbance',
    ),
    yaxis=dict(
        title='Intensity Fluorescence (a.u.)'
    ),
    title='Intensity Fluorescence vs Absorbance',
    height=600,
    width=800,
    legend=dict(x=0, y=1),
    font=dict(size=14)
)

fig.show()

In [12]:


# iteration 0
popt, pcov = curve_fit(F_C_fit, A, I, sigma=sigmaI, maxfev=10000)
F0, k, y0 = popt
error_F0, error_k, error_y0 = np.sqrt(np.diag(pcov))
print(F0, k, y0)

# iteration 1
dFdC = np.abs(k * F0 * np.exp(-k * A)) # propagate error on C
sigmaTot = np.sqrt(sigmaI**2 + (dFdC * sigmaA)**2) # propagate error on I
popt, pcov = curve_fit(F_C_fit, A, I, p0=[F0, k, y0], sigma=sigmaTot)
F0, k, y0 = popt
error_F0, error_k, error_y0 = np.sqrt(np.diag(pcov))

print('F0:', final_val(F0, error_F0, decimals=0, udm='a.u.'))
print('k:', final_val(k, error_k, decimals=3))
print('y0:', final_val(y0, error_y0, decimals=3, udm='a.u.'))

# Create a scatter plot with error bars using Plotly
fig = go.Figure()

# Add data points with error bars
fig.add_trace(go.Scatter(
    x=A,
    y=I,
    error_y=dict(
        type='data',
        array=sigmaI,
        visible=True
    ),
    error_x=dict(
        type='data',
        array=sigmaA,
        visible=True
    ),
    mode='markers',
    name='Data with error'
))

# Add the fitted curve
a = np.linspace(np.min(A), np.max(A), 1000)
fig.add_trace(go.Scatter(
    x=a,
    y=F_C_fit(a, *popt),
    mode='lines',
    name='Fit'
))

# Update layout for log scale and labels
fig.update_layout(
    xaxis=dict(
        title='Absorbance',
       # type='log'
    ),
    yaxis=dict(
        title='Intensity Fluorescence (a.u.)'
    ),
    # title='Intensity Fluorescence vs Concentration',
    height=600,
    width=800,
    legend=dict(x=0, y=1),
    font=dict(size=14)
)

fig.write_html(dir_path +r"\html\FvsAfit.html")
fig.write_image(dir_path + r"\images\FvsAfit.png")
fig.show()

1364.5828491148986 3.1255776403536792 -90.3792499123716
F0: 1333 ± 77 a.u.
k: 3.296 ± 0.351 
y0: -95.611 ± 9.928 a.u.


In [13]:
print(A)
print(sigmaA)

[0.48004168 0.3526902  0.2634356  0.17474998 0.08754881 0.07047534
 0.05284477 0.03526507]
[3.35343104e-04 3.16099377e-04 2.77603691e-04 2.11906585e-04
 1.24380863e-04 1.06860277e-04 8.53311349e-05 6.07545935e-05]


In [14]:
# fit lineare del primo set di dati:
limit = 0.1
def F_C_fit_linear(C,k,y0):
    return k*C + y0

I2 = I[A < limit]
sigmaI2 =sigmaI[A < limit] 

A2 = A[A <limit] 
sigmaA2 = sigmaA[A < limit]

# iteration 0
popt, pcov = curve_fit(F_C_fit_linear, A2, I2, p0=[1000,1.0],sigma=sigmaI2,maxfev=10000)
k, y0 = popt
error_k, error_y0 = np.sqrt(np.diag(pcov))

# iteration 1
dFdC = np.abs(k*F0*np.exp(-k*A2)) # propagate error on C
sigmaTot = np.sqrt(sigmaI2**2 + (dFdC*sigmaA2)**2) # propagate error on I
popt, pcov = curve_fit(F_C_fit_linear, A2, I2, p0=[k,y0],sigma=sigmaTot)
k, y0 = popt
error_k, error_y0 = np.sqrt(np.diag(pcov))


# print('F0:',final_val(F0,error_F0,decimals=0,udm='a.u.'))
print('k:',final_val(k,error_k,decimals=3,udm='μM^-1'))
print('y0:',final_val(y0,error_y0,decimals=3,udm='a.u.'))

# Create a scatter plot with error bars using Plotly
fig = go.Figure()

# Add data points with error bars
fig.add_trace(go.Scatter(
    x=A2,
    y=I2,
    error_y=dict(
        type='data',
        array=sigmaI2,
        visible=True
    ),
    error_x=dict(
        type='data',
        array=sigmaA2,
        visible=True
    ),
    mode='markers',
    name='Data with error'
))

# Add the fitted curve
a = np.linspace(np.min(A2), np.max(A2), 1000)
fig.add_trace(go.Scatter(
    x=a,
    y=F_C_fit_linear(a, *popt),
    mode='lines',
    name='Fit'
))

# Update layout for log scale and labels
fig.update_layout(
    xaxis=dict(
        title='Concentration (μM)',
        # type='log'
    ),
    yaxis=dict(
        title='Intensity Fluorescence (a.u.)'
    ),
    title='Intensity Fluorescence vs Concentration',
    height=600,
    width=800,
    legend=dict(x=0, y=1),
    font=dict(size=14)
)

fig.write_html(dir_path +r"\html\FvsAfit_linear.html")
fig.write_image(dir_path + r"\images\FvsAfit_lnear.png")
fig.show()


k: 3493.111 ± 110.065 μM^-1
y0: -70.34 ± 4.899 a.u.
